# Load libraries

In [1]:
### Load libraries 
import os
import numpy as np
import pandas as pd

#import sqlite3 as sl
import uuid
#import csv
#import os.path
#from os import path
import glob
#import time
import re
from datetime import datetime

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from ipywidgets import *
from IPython.display import clear_output, display
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import qgrid

import helpers.research_helpers as app

CSV_SEPARATOR = ','     # '\t'
PIPE_SEPARATOR = '|#' 

TBL_SURVEYS = 'SURVEYS'
TBL_MEMBERS = 'MEMBERS'
TBL_MEMBERS_TO_SURVEYS = 'MEMBERS_TO_SURVEYS'

SURVEYS_COL_SURVEY_ID = 'survey_id'
SURVEYS_COL_NAME = 'name'
SURVEYS_COL_DATE = 'date'

MEMBERS_COL_SURVEY_ID = 'member_id'
MEMBERS_COL_FNAME = 'first_name'
MEMBERS_COL_LNAME = 'last_name'
MEMBERS_COL_EMAIL = 'email'
MEMBERS_COL_LAST_SURVEY_DATE = 'last_survey_date'

print('Notebook initialized.')

Notebook initialized.


In [2]:
app.FLDR_SAMPLING

'../sampling'

In [ ]:
df_complete_list = pd.read_csv (app.FLDR_SAMPLING + '/AllActive/AllActiveMem.csv' , sep=PIPE_SEPARATOR)

In [ ]:
columns = ['member_id', 'expiration_date', 'name_prefix', 'first_name', 'middle_name', 'last_name', 'name_suffix', 
           'cert_type', 'cert_start_date', 'cert_end_date', 'job_title_freetext', 'companyname', 
           'shipaddr_1', 'shipaddr_2', 'shipaddr_city', 'shipaddr_state', 'shipaddr_zip', 
           'shipaddr_country', 'shipaddr_country_desc', 'shipaddr_type', 
           'email', 'disallow_all_communication', 'disallow_email_communication', 'disallow_phone_communication', 
           'disallow_regular_mail_communication', 'disallow_third_party_communication', 
           'job_title', 'job_function', 'employment_status', 'company_size', 'department_size', 
           'industry_category', 'organization_unit', 'organization_type', 'employee_oversee', 'supervisor_title', 
           'gender', 'ethnicity', 'education', 'native_language', 'customer_segment', 'title_calculated', 
           'gender_calculated', 'education_calculated', 'age_calculated', 'ethnicity_calculated', 
           'state_calculated', 'location_calculated', 'region_calculated', 'pub_private_calculated', 
           'sample_size_calculated', 'time_zone_calculated']
df_trimmed_list = df_complete_list[columns]

# Load All Active Roster.csv

In [ ]:
### Load All Active Roster.csv 

#df_complete_list = app.load_csv(app.FLDR_SAMPLING + '/All Active Roster.csv')
df_complete_list = pd.read_csv (app.FLDR_SAMPLING + '/All Active Roster.csv' , sep=CSV_SEPARATOR)
df_complete_list.columns = df_complete_list.columns.str.lower()

df_complete_list = df_complete_list.rename(columns=str.lower)
df_complete_list = df_complete_list[['member_id','email', 'firstname', 'lastname', 'job_title']]
df_complete_list['email'] = df_complete_list['email'].str.lower()
df_complete_list['email'] = df_complete_list['email'].str.strip()

df_complete_list['member_id'] = df_complete_list['member_id'].astype('string')
df_complete_list['email'] = df_complete_list['email'].astype('string')
df_complete_list['firstname'] = df_complete_list['firstname'].astype('string')
df_complete_list['lastname'] = df_complete_list['lastname'].astype('string')
df_complete_list['job_title'] = df_complete_list['job_title'].astype('string')
#df_complete_list['last_survey_date'] = df_complete_list['last_survey_date'].astype('category')

print(str(len(df_complete_list)) + ' records loaded from ' + app.FLDR_SAMPLING + '/All Active Roster.csv')

# Determine which survey directories should be saved to DB

In [ ]:
### OLD - Search file system for survey directories needing to be processed. 
if False:
    if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS.csv'):
        df_surveys = pd.read_csv (app.FLDR_SAMPLING + '/SURVEYS.csv' , sep=CSV_SEPARATOR, keep_default_na=False)
        df_surveys.columns= df_surveys.columns.str.lower()
    else:
        df_surveys = pd.DataFrame(columns = ['survey_id','name','date'])
    df_surveys['new_entry'] = 0

    def get_date_name (row):
        new_val = str(row['date']) + '___' + str(row['name'])
        return new_val

    def get_unprocessed_survey_dirs(df_saved_surveys):
        # get list of processed survey dirs

        surveys_dict = {}
        print('Count of previously processed survey dirs: {}'.format(str(len(df_saved_surveys['name'].values))))

        for index, row in df_saved_surveys.iterrows(): 
            surveys_dict[row[SURVEYS_COL_NAME]] = row[SURVEYS_COL_DATE]

        # find potential unprocessed survey dirs by iterating over folder names in s_drive and, 
        # comparing names to previously processed list
        df = pd.DataFrame(columns=['name','date'])

        for filename in glob.iglob(app.FLDR_SAMPLING + '/**/*.[cC][sS][vV]', recursive=True):
            if 'qualtrics' in filename.lower():

                match_date = re.search(r'\d{4}-\d{2}-\d{2}', filename.split('/')[-1])
                if match_date != None:
                    survey_date = datetime.strptime(match_date.group(), '%Y-%m-%d').date()
                else:
                    t = os.path.getmtime(filename)
                    survey_date = datetime.fromtimestamp(t)

                survey_name = filename.split('/')[-2]

                # verify survey_name starts with a year AND hasn't already been processed
                if survey_name not in surveys_dict:
                    surveys_dict[survey_name] = survey_date
                    #print('{}, on {}'.format(survey_name, survey_date.strftime('%Y-%m-%d')))
                    new_row = {'name':survey_name, 'date': survey_date.strftime('%Y-%m-%d')}
                    df = df.append(new_row, ignore_index=True)

        if len(df) > 0: # found new records, append to db
            return df
        else:
            df = pd.DataFrame(columns = [SURVEYS_COL_NAME, SURVEYS_COL_DATE])
            return df

    ### Get a dataframe of surveys that have not been processed
    df_unprocessed_survey_dirs = get_unprocessed_survey_dirs(df_surveys)
    #print('Count of unprocessed survey dirs: {}'.format(str(len(df_unprocessed_survey_dirs))))
    #if df_unprocessed_survey_dirs is None:
    #    print('There are no new survey directories to process :)')
    #else:
    #    # Allow user's to select which surveys, if any, they want to exclude
    #    qgrid_sheet_to_keep = app.keep_grid_show_filter(df_unprocessed_survey_dirs, 'name', None, app.KEEP_LIST_SURVEY_DIRS)

In [ ]:
### Search file system for survey directories needing to be processed. 

if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS.csv'):
    df_surveys = pd.read_csv (app.FLDR_SAMPLING + '/SURVEYS.csv' , sep=CSV_SEPARATOR, keep_default_na=False)
    df_surveys.columns= df_surveys.columns.str.lower()
else:
    df_surveys = pd.DataFrame(columns = ['survey_id','name','date'])
df_surveys['new_entry'] = 0

def get_date_name (row):
    new_val = str(row['date']) + '___' + str(row['name'])
    return new_val

def get_unprocessed_survey_dirs(df_saved_surveys):
    # get list of processed survey dirs
    
    surveys_dict = {}
    print('Count of previously processed survey dirs: {}'.format(str(len(df_saved_surveys['name'].values))))
    
    for index, row in df_saved_surveys.iterrows(): 
        surveys_dict[row[SURVEYS_COL_NAME]] = row[SURVEYS_COL_DATE]

    # find potential unprocessed survey dirs by iterating over folder names in s_drive and, 
    # comparing names to previously processed list
    df = pd.DataFrame(columns=['name','date'])
    
    for folder in glob.iglob(app.FLDR_SAMPLING + '/**/', recursive=False):
        survey_name = folder.split('/')[2]
        #print(survey_name)
        if 'notebooks' not in survey_name and 'Sampling R Code' not in survey_name and survey_name not in surveys_dict:
            match_date = re.search(r'\d{4}-\d{2}-\d{2}', folder.split('/')[-1])
            if match_date != None:
                survey_date = datetime.strptime(match_date.group(), '%Y-%m-%d').date()
            else:
                t = os.path.getmtime(folder)
                survey_date = datetime.fromtimestamp(t)

            surveys_dict[survey_name] = survey_date
            #print('{}, on {}'.format(survey_name, survey_date.strftime('%Y-%m-%d')))
            new_row = {'name':survey_name, 'date': survey_date.strftime('%Y-%m-%d')}
            df = df.append(new_row, ignore_index=True)

    if len(df) > 0: # found new records, append to db
        return df
    else:
        df = pd.DataFrame(columns = [SURVEYS_COL_NAME, SURVEYS_COL_DATE])
        return df

### Get a dataframe of surveys that have not been processed
df_unprocessed_survey_dirs = get_unprocessed_survey_dirs(df_surveys)

In [ ]:
### Please select the survey directories you would like to process. 

print('Count of unprocessed survey dirs: {}'.format(str(len(df_unprocessed_survey_dirs))))
if df_unprocessed_survey_dirs is None:
    print('There are no new survey directories to process :)')
else:
    # Allow user's to select which surveys, if any, they want to exclude
    qgrid_sheet_to_keep = app.keep_grid_show_filter(df_unprocessed_survey_dirs, 'name', None, app.KEEP_LIST_SURVEY_DIRS)

## Apply changes

In [ ]:
### Apply filtering 
#
# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list_of_survey_dirs, app.KEEP_LIST_SURVEY_DIRS = app.keep_grid_apply_filter(qgrid_sheet_to_keep, 'name', df_unprocessed_survey_dirs, True)


# Process survey information in selected directories

In [ ]:
### Iterate over directories and process survey information 

df_post_merge = None
df_current_dir = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])
df_surveyed_members_to_processed = None

for index, row in df_filtered_list_of_survey_dirs.iterrows(): 
    df_concatenated = pd.DataFrame(columns = ['email', 'last_survey_date', 'member_id', 'firstname', 'lastname', 'job_title'])

    print('folder  name: ' + row['name'])
    df_current_file = None
    df_post_merge = None
    df_current_dir = None
    #df_current_dir = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])
    
    survey_id = len(df_surveys.index)
    #df_surveys['new_entry'] = 0
    survey_dir = {}
    survey_dir = {'survey_id': survey_id, 'name': row['name'], 'date': row['date'], 'new_entry': 1, }
    df_surveys = df_surveys.append(survey_dir, ignore_index = True)
    
    for filename in glob.iglob(app.FLDR_SAMPLING + '/' + row['name'] + '/**/*.[cC][sS][vV]', recursive=True):
        if 'qualtrics' in filename.lower():
            print('  -- ' + filename)
            
            # read csv from directory
            #df_current_file = None
            df_current_file = pd.read_csv(filename)
            df_current_file = df_current_file.rename(columns=str.lower)
            df_current_file['last_survey_date'] = row['date']
            df_current_file['survey_name'] = row['name']
            df_current_file['survey_id'] = survey_id
            df_current_file['email'] = df_current_file['email'].str.lower()
            df_current_file['email'] = df_current_file['email'].str.strip()
            df_current_file = df_current_file.rename({'fname' : 'first_name'}, axis=1)
            df_current_file = df_current_file.rename({'firstname' : 'first_name'}, axis=1)
            df_current_file = df_current_file.rename({'lname' : 'last_name'}, axis=1)
            df_current_file = df_current_file.rename({'lastname' : 'last_name'}, axis=1)
            #df_current_file = df_current_file.drop(['first_name', 'last_name'], axis=1)
            
            #df_current_file.head()
            #print('  -- survey respondents loaded from CSV: {}'.format(str(len(df_current_file))))
            #df_current_file.sort_values('email', inplace = True)
            df_current_file.drop_duplicates(subset ='email', keep = 'first', inplace = True)
            #print('  -- survey respondents (after dropping dups): {}'.format(str(len(df_current_file))))
            
            
            #merge current CSV with other CSVs processed this session
            df_surveyed_members_to_processed = pd.concat([df_surveyed_members_to_processed, df_current_file], axis=0)
            print('  -- df_surveyed_members_to_processed {}'.format(str(len(df_surveyed_members_to_processed))))
            
#df_surveyed_members_to_processed.head()
#print('df_surveyed_members_to_processed {}'.format(str(len(df_surveyed_members_to_processed))))

# Save to db

In [ ]:
### Merge with All Active Roster to get true names and member_ids 

def __create_missing_member_id(member_id):
    str_member_id = str(member_id)
    
    if len(str_member_id) == 0:
        str_member_id = uuid.uuid4()
    
    return str_member_id

def __reassign_member_id(member_id, member_id_y):
    str_member_id = str(member_id)
    
    if len(member_id_y) > 0:
        str_member_id = member_id_y
    
    return str_member_id

def merge_with_all_active_roster(df_left, df_right):
    df = None
    df = pd.merge(left=df_left, right=df_right, how='left', left_on='email', right_on='email')
    df.drop_duplicates(subset =['survey_id', 'email'], inplace = True)
    df['firstname'] = df['firstname'].fillna('')
    df['lastname'] = df['lastname'].fillna('')
    df['job_title'] = df['job_title'].fillna('')
    df['member_id'] = df['member_id'].fillna('')
    df['member_id'] = df.apply(lambda x: __create_missing_member_id(str(x['member_id']).strip()), axis=1)     
    df['firstname'] = df['first_name']
    df['lastname'] = df['last_name']
    
    print('  -- merge_with_all_active_roster: {}'.format(str(len(df))))
    
    return df

def assign_proper_member_id(df):
    df.sort_values(['email', 'last_survey_date'], inplace = True, ascending = [True, False])
    df_post_merge_unique_member_id = None
    df_post_merge_unique_member_id = df.drop_duplicates(subset =['email'], inplace = False, keep = 'first')
    df_post_merge_unique_member_id = df_post_merge_unique_member_id[['email', 'member_id']]
    df = pd.merge(left=df_post_merge_unique_member_id, right=df, how='left', left_on='email', right_on='email')
    df = df.rename(columns={'member_id_x': 'member_id'})
    df = df[['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date', 'survey_name', 'survey_id']]
    df.sort_values(['email', 'last_survey_date'], inplace = True, ascending = [True, False])

    print('  -- assign_proper_member_id: {}'.format(str(len(df))))
    return df

def assign_autogenerated_member_id_from_prev_survey(df_left):
    if os.path.exists(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv'):
        df_right = pd.read_csv (app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv' , sep=CSV_SEPARATOR, keep_default_na=False)
        df_right.columns= df_right.columns.str.lower()
    else:
        df_right = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])

    if len(df_right) > 0:
        print('  -- records loaded from ' + app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv: ' + str(len(df_right)))
        
        df = None
        df = pd.merge(left=df_left, right=df_right, how='left', left_on='email', right_on='email')
        df = df.rename(columns={'member_id_x': 'member_id'})
        df = df.rename(columns={'firstname_x': 'firstname'})
        df = df.rename(columns={'lastname_x': 'lastname'})
        df = df.rename(columns={'job_title_x': 'job_title'})
        df = df.rename(columns={'last_survey_date_x': 'last_survey_date'})
        df['member_id_y'] = df['member_id_y'].fillna('')

        df['member_id'] = df.apply(lambda x: __reassign_member_id(str(x['member_id']).strip(), str(x['member_id_y']).strip()), axis=1)   

        #df_post_merge2.head(n=50)
        df = df[['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date', 'survey_name', 'survey_id']]
        df.sort_values(['email', 'last_survey_date'], inplace = True, ascending = [True, False])

        print('  -- assign_autogenerated_member_id_from_prev_survey: {}'.format(str(len(df))))
        return df, df_right
    else:
        print('  -- assign_autogenerated_member_id_from_prev_survey: {}'.format(str(len(df_left))))
        return df_left, df_right

if df_surveyed_members_to_processed is not None:
    df_post_merge = None
    df_post_merge = merge_with_all_active_roster(df_surveyed_members_to_processed, df_complete_list)
    #df_post_merge.head()

    df_post_merge = assign_proper_member_id(df_post_merge)
    #df_post_merge.head(n=50)

    df_post_merge, df_surveyed_members = assign_autogenerated_member_id_from_prev_survey(df_post_merge)
    #df_post_merge.head(n=50)


In [ ]:
### Process df to determine surveys to save to app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv' 

if df_surveyed_members_to_processed is not None:
    df_surveys_to_members_temp = None
    df_surveys_to_members_temp = df_post_merge[['survey_id', 'member_id', 'email']]
    df_surveys_to_members_temp.sort_values(['survey_id', 'email'], inplace = True)
    df_surveys_to_members_temp.drop_duplicates(subset =['survey_id', 'email'], inplace = True)
    df_surveys_to_members_temp = df_surveys_to_members_temp[['survey_id', 'member_id']]
    len(df_surveys_to_members_temp)
    #df_surveys_to_members_temp = pd.concat([df_surveys_to_members_temp, df_surveys_to_members], axis=0)
    #df_surveys_to_members_temp.sort_values(['survey_id', 'member_id'], inplace = True)
    #df_surveys_to_members_temp.drop_duplicates(subset = ['survey_id', 'member_id'], keep = 'first', inplace = True)
    #df_surveys_to_members = df_surveys_to_members_temp

    if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv'):
        df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'a', header = False, index=False, quotechar='"')
    else:
        df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
        #df_surveys_to_members_temp.head()

    #df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
    print('  -- df_surveys_to_members_temp: {}'.format(str(len(df_surveys_to_members_temp))))

In [ ]:
### Process df to save to app.FLDR_SAMPLING + '/SURVEYS.csv' 
    df_surveys_temp = df_surveys[df_surveys['new_entry']==1]
    df_surveys_temp = df_surveys_temp[['survey_id', 'name', 'date']]

    if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS.csv'):
        df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'a', header = False, index=False, quotechar='"')
    else:
        df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'w', header = True, index=False, quotechar='"')

    df_surveys['new_entry'] = 0
    #df_surveys.head()
    print('  -- df_surveys_temp: {}'.format(str(len(df_surveys_temp))))

In [ ]:
### Process df to save to app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv' 
if df_surveyed_members_to_processed is not None:
    df_surveyed_members_temp = None
    df_surveyed_members_temp = df_post_merge[['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date']]
    df_surveyed_members_temp.sort_values(['email','last_survey_date'], inplace = True)
    df_surveyed_members_temp.drop_duplicates(subset =['email'], inplace = True)

    df_surveyed_members_temp = pd.concat([df_surveyed_members_temp, df_surveyed_members], axis=0)
    df_surveyed_members_temp.sort_values(['email','last_survey_date'], inplace = True)
    df_surveyed_members_temp.drop_duplicates(subset =['email'], inplace = True)
    df_surveyed_members = df_surveyed_members_temp

    #if os.path.exists(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv'):
    #    df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv', mode = 'a', header = Fa, index=False, quotechar='"')
    #else:
    #    df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
    #    #df_surveyed_members_temp.head()

    df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
    print('  -- df_surveyed_members_temp: {}'.format(str(len(df_surveyed_members_temp))))

In [ ]:
### Show number of surveys left to process 
processed_surveys = df_filtered_list_of_survey_dirs['name'].values
df_unprocessed_survey_dirs = df_unprocessed_survey_dirs[~df_unprocessed_survey_dirs['name'].isin(processed_surveys)]

if len(df_unprocessed_survey_dirs) == 0:
    df_unprocessed_survey_dirs = None

print('Number of survey directories left to process: {}'.format(str(len(df_unprocessed_survey_dirs))))